In [1]:
import pandas as pd
import numpy as np

# Реальные средние температуры (примерные данные) для городов по сезонам
seasonal_temperatures = {
    "New York": {"winter": 0, "spring": 10, "summer": 25, "autumn": 15},
    "London": {"winter": 5, "spring": 11, "summer": 18, "autumn": 12},
    "Paris": {"winter": 4, "spring": 12, "summer": 20, "autumn": 13},
    "Tokyo": {"winter": 6, "spring": 15, "summer": 27, "autumn": 18},
    "Moscow": {"winter": -10, "spring": 5, "summer": 18, "autumn": 8},
    "Sydney": {"winter": 12, "spring": 18, "summer": 25, "autumn": 20},
    "Berlin": {"winter": 0, "spring": 10, "summer": 20, "autumn": 11},
    "Beijing": {"winter": -2, "spring": 13, "summer": 27, "autumn": 16},
    "Rio de Janeiro": {"winter": 20, "spring": 25, "summer": 30, "autumn": 25},
    "Dubai": {"winter": 20, "spring": 30, "summer": 40, "autumn": 30},
    "Los Angeles": {"winter": 15, "spring": 18, "summer": 25, "autumn": 20},
    "Singapore": {"winter": 27, "spring": 28, "summer": 28, "autumn": 27},
    "Mumbai": {"winter": 25, "spring": 30, "summer": 35, "autumn": 30},
    "Cairo": {"winter": 15, "spring": 25, "summer": 35, "autumn": 25},
    "Mexico City": {"winter": 12, "spring": 18, "summer": 20, "autumn": 15},
}

# Сопоставление месяцев с сезонами
month_to_season = {12: "winter", 1: "winter", 2: "winter",
                   3: "spring", 4: "spring", 5: "spring",
                   6: "summer", 7: "summer", 8: "summer",
                   9: "autumn", 10: "autumn", 11: "autumn"}

# Генерация данных о температуре
def generate_realistic_temperature_data(cities, num_years=10):
    dates = pd.date_range(start="2010-01-01", periods=365 * num_years, freq="D")
    data = []

    for city in cities:
        for date in dates:
            season = month_to_season[date.month]
            mean_temp = seasonal_temperatures[city][season]
            # Добавляем случайное отклонение
            temperature = np.random.normal(loc=mean_temp, scale=5)
            data.append({"city": city, "timestamp": date, "temperature": temperature})

    df = pd.DataFrame(data)
    df['season'] = df['timestamp'].dt.month.map(lambda x: month_to_season[x])
    return df

# Генерация данных
data = generate_realistic_temperature_data(list(seasonal_temperatures.keys()))
data.to_csv('temperature_data.csv', index=False)


In [2]:
data = pd.read_csv('temperature_data.csv')

In [3]:
data.head(5)

,city,timestamp,temperature,season
0,New York,2010-01-01,-6.172187,winter
1,New York,2010-01-02,-3.946836,winter
2,New York,2010-01-03,-10.446251,winter
3,New York,2010-01-04,-0.910715,winter
4,New York,2010-01-05,-8.812923,winter


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54750 entries, 0 to 54749
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         54750 non-null  object 
 1   timestamp    54750 non-null  object 
 2   temperature  54750 non-null  float64
 3   season       54750 non-null  object 
dtypes: float64(1), object(3)
memory usage: 1.7+ MB


In [5]:
data.describe()

,temperature
count,54750.000000
mean,18.273296
std,11.023147
min,-23.611717
25%,11.293607
50%,18.723461
75%,26.097295
max,55.345660


In [6]:
data.describe(include='object')

,city,timestamp,season
count,54750,54750,54750
unique,15,3650,4
top,New York,2010-01-01,spring
freq,3650,15,13800


In [7]:
from multiprocessing import Pool
from functools import partial
import time
from sklearn.linear_model import LinearRegression

# Загрузка данных
data = pd.read_csv('temperature_data.csv', parse_dates=['timestamp'])

In [8]:
# Функция для анализа данных по городу
def analyze_city(city_data):
    city = city_data['city'].iloc[0]

    # Вычисление скользящего среднего
    city_data = city_data.sort_values('timestamp')
    city_data['rolling_avg'] = city_data['temperature'].rolling(window=30, min_periods=1).mean()
    city_data['rolling_std'] = city_data['temperature'].rolling(window=30, min_periods=1).std()

    # Выявление аномалий на основе скользящего окна
    city_data['is_anomaly_window'] = (
        (city_data['temperature'] < city_data['rolling_avg'] - 2 * city_data['rolling_std']) |
        (city_data['temperature'] > city_data['rolling_avg'] + 2 * city_data['rolling_std'])
    )

    # Расчет среднего и стандартного отклонения для каждого сезона
    mean_value = city_data.groupby('season')['temperature'].mean().reset_index()
    std_value = city_data.groupby('season')['temperature'].std().reset_index()
    stats = pd.merge(mean_value, std_value, on='season', suffixes=('_mean', '_std'))

    # Выявление аномалий
    def find_anomalies(row):
        season_stats = stats[stats['season'] == row['season']]
        mean = season_stats['temperature_mean'].values[0]
        std = season_stats['temperature_std'].values[0]
        if row['temperature'] < mean - 2 * std or row['temperature'] > mean + 2 * std:
            return True
        return False

    city_data['is_anomaly'] = city_data.apply(find_anomalies, axis=1)

    # Тренд (линейная регрессия)
    city_data['timestamp_ordinal'] = city_data['timestamp'].map(pd.Timestamp.toordinal)
    lr = LinearRegression()
    lr.fit(city_data[['timestamp_ordinal']], city_data['temperature'])
    trend_slope = lr.coef_[0]
    trend = 'positive' if trend_slope > 0 else 'negative'

    # Средняя, min, max температуры за всё время
    avg_temp = city_data['temperature'].mean()
    min_temp = city_data['temperature'].min()
    max_temp = city_data['temperature'].max()

    return {
        'city': city,
        'avg_temp': avg_temp,
        'min_temp': min_temp,
        'max_temp': max_temp,
        'season_profile': stats,
        'trend': trend,
        'anomalies': city_data[city_data['is_anomaly_window']]
    }

In [9]:
# Анализ без распараллеливания
start = time.time()
results_non_parallel = [analyze_city(group) for _, group in data.groupby('city')]
end = time.time()
non_parallel_time = end - start
print(f"Время выполнения без распараллеливания: {non_parallel_time:.2f} секунд")

Время выполнения без распараллеливания: 28.36 секунд


In [10]:
# Анализ с распараллеливанием
start = time.time()
with Pool() as pool:
    result_parallel = pool.map(analyze_city, [group for _, group in data.groupby('city')])
end = time.time()
parallel_time = end - start
print(f"Время выполнения с распараллеливанием: {parallel_time:.2f} секунд")

Время выполнения с распараллеливанием: 21.02 секунд


In [11]:
import requests
import pandas as pd
import asyncio
import aiohttp

# OpenWeatherMap
api_key = 'api_key'
url = 'https://api.openweathermap.org/data/2.5/weather'

In [12]:
# Функция для получения текущей температуры (синхронный способ)
def get_current_temperature_sync(city):
    params = {'q': city, 'appid': api_key, 'units': 'metric'}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data['main']['temp']
    else:
        print(f"Ошибка при запросе для {city}: {response.status_code}")
        return None

In [13]:
# Функция для получения текущей температуры (асинхронный способ)
async def get_current_temperature_async(city, session):
    params = {'q': city, 'appid': api_key, 'units': 'metric'}
    async with session.get(url, params=params) as response:
        if response.status == 200:
            data = await response.json()
            return data['main']['temp']
        else:
            print(f"Ошибка при запросе для {city}: {response.status}")
            return None

In [14]:
# Функция для проверки нормальности температуры
def check_temperature_normality(city, current_temp, stats):
    season_stats = stats[stats['city'] == city]
    mean = season_stats['temperature_mean'].values[0]
    std = season_stats['temperature_std'].values[0]
    if current_temp < mean - 2 * std or current_temp > mean + 2 * std:
        return "Аномальная температура"
    else:
        return "Температура в норме"

In [15]:
# Данные для тестирования
cities = ["Berlin", "Cairo", "Dubai", "Beijing", "Moscow"]

# Пример статистики (в реальном коде это заменяется на данные из анализа)
stats = pd.DataFrame({
    'city': ["Berlin", "Cairo", "Dubai", "Beijing", "Moscow"],
    'temperature_mean': [10, 25, 30, -5, 0],
    'temperature_std': [5, 3, 5, 7, 8]
})

In [20]:
# Пример использования (синхронный способ)
def monitor_temperature_sync(cities, stats):
    for city in cities:
        current_temp = get_current_temperature_sync(city)
        if current_temp is not None:
            print(f"Текущая температура в {city}: {current_temp}°C")
            status = check_temperature_normality(city, current_temp, stats)
            print(f"Статус: {status}")

print("\nСинхронный способ")
monitor_temperature_sync(cities, stats)


Синхронный способ
Текущая температура в Berlin: 5.58°C
Статус: Температура в норме
Текущая температура в Cairo: 17.42°C
Статус: Аномальная температура
Текущая температура в Dubai: 19.96°C
Статус: Аномальная температура
Текущая температура в Beijing: -4.06°C
Статус: Температура в норме
Текущая температура в Moscow: -2.76°C
Статус: Температура в норме


In [21]:
import nest_asyncio
nest_asyncio.apply()

# Пример использования (асинхронный способ)
async def monitor_temperature_async(cities, stats):
    async with aiohttp.ClientSession() as session:
        tasks = [get_current_temperature_async(city, session) for city in cities]
        temperatures = await asyncio.gather(*tasks)

        for city, current_temp in zip(cities, temperatures):
            if current_temp is not None:
                print(f"Текущая температура в {city}: {current_temp}°C")
                status = check_temperature_normality(city, current_temp, stats)
                print(f"Статус: {status}")

print("\nАсинхронный способ")
await monitor_temperature_async(cities, stats)


Асинхронный способ
Текущая температура в Berlin: 5.58°C
Статус: Температура в норме
Текущая температура в Cairo: 17.42°C
Статус: Аномальная температура
Текущая температура в Dubai: 19.96°C
Статус: Аномальная температура
Текущая температура в Beijing: -4.06°C
Статус: Температура в норме
Текущая температура в Moscow: -2.76°C
Статус: Температура в норме
